In [0]:
%pyspark
#All imports go here
from shapely.geometry import shape
from shapely.ops import transform
from shapely.geometry import MultiPolygon
from shapely import wkt
from shapely.ops import transform

from functools import partial
import pyproj

import pyspark.sql.functions as pys
from pyspark.sql.types import DoubleType, StringType, FloatType, IntegerType, BooleanType
from pyspark.sql.functions import udf
from pyspark.sql.functions import sum
from pyspark.sql.functions import col

import plotly.plotly as py
import plotly.graph_objs as go
import pandas as pd

In [0]:
%pyspark

#Function to convert MultiPoly to Area in Acre
def getAreaOfMultiPoly(columnElement):
    try:
        s = unicode(str(columnElement), "utf-8")
        p = wkt.loads(s)
        m = MultiPolygon(p)
        
        project = partial(
        pyproj.transform,
        pyproj.Proj(init='epsg:4326'), # source coordinate system
        pyproj.Proj(init='epsg:26913')) # destination coordinate system

        m2 = transform(project, m)
        return m2.area/4046.86
    except:
        return 0
    
#Check if field contains non empty boundary data
def isBoundaryPresent(columnElement):
    if not columnElement:
        return False
    try:
        
        columnElement = columnElement.strip()
        columnElement = unicode(str(columnElement), "utf-8")
        
        if columnElement == "MULTIPOLYGON EMPTY" or columnElement == None or columnElement == "null" or columnElement == "" or columnElement == " " or  columnElement == None:
            return False
            
        
        if columnElement[:5] == "MULTI":
            return True
            
        return False
        
            
            
    except:
        print "OOOOps"
        return False
    
    

def calculateDifferencePercentageInArea(area, areaCalculated):
    try:
        return (area-areaCalculated)*100/areaCalculated
    except:
        return 0


def convertAreaToAcres(area, area_unit):
    try:
        area = float(area)
        if area == "None" or area == "none" or area == None:
            return 0
        if area_unit == None or area_unit == 'null' or area_unit == "":
            return 0
        if (area_unit) == 'm^2':
            return (area/4046)
            
        if (area_unit) == 'sqft':
            return ((area)/(43560))
            
        if (area_unit) == 'a':
            return (area)
            
            
        return 0
    except:
        return 0


def getDigitVal(value):
    value = str(value)
    if not value:
        return "0"
        
    if value.isdigit:
        return value
        
    return "0"


convertAreaUDF = udf(lambda x, y: convertAreaToAcres(x, str(y)), DoubleType())

is_digit_udf = udf(lambda z: getDigitVal(z), StringType())
getAreaOfMultiPoly_udf = udf(lambda z: getAreaOfMultiPoly(z), DoubleType())
getAreaDifferencePercentage = udf(lambda (x,y): calculateDifferencePercentageInArea(float(x),float(y)), DoubleType())
checkIfBoundaryPresent = udf(lambda z: isBoundaryPresent(z), BooleanType())



In [0]:
%pyspark

fields = sqlContext.read.json("NonExploded_New_data_field.json/*.json")
fields = fields.where(col('field.deleted_at').isNull())

fields2 = fields.select('field')


In [0]:
%pyspark
fields2.select('field.area_unit').distinct().show()

In [0]:
%pyspark
fieldsNotNull =  fields2.filter(col("field.area_unit").isNotNull())


In [0]:
%pyspark
fieldsFiltered = fieldsNotNull.select('field.id', 'field.area', 'field.area_unit', 'field.associations.`agrian.farm`', 'field.associations.`agrian.grower`', 'field.boundary_map.boundary', 'field.state_id')\
.toDF('id', 'area', 'area_unit', 'farm_id', 'grower_id', 'boundary', 'state_id')

In [0]:
%pyspark
processedJson = fieldsFiltered.withColumn('area', is_digit_udf(fieldsFiltered['area']))
processedJson = processedJson.withColumn('isBoundaryPresent', checkIfBoundaryPresent(processedJson.boundary))
processedJson.show()

In [0]:
%pyspark
#processedJson.select('isBoundaryPresent').distinct().show()

In [0]:
%pyspark
processedJson.printSchema()
processedJson.show()

In [0]:
%pyspark
temp = processedJson
processedJson2 = temp.withColumn('areaConverted', pys.abs(convertAreaUDF(temp.area, temp.area_unit )))


In [0]:
%pyspark
processedJson2.show()

In [0]:
%pyspark
#processedJson2.where("area == 0").count()
processedJson3 = processedJson2.where("area != 0")

In [0]:
%pyspark
total = processedJson3.withColumn("areaCalculated",getAreaOfMultiPoly_udf("boundary"))
total = total.withColumn("areaCalculated", pys.abs(total.areaCalculated.cast('float')))
#total = total.withColumn('area_differnce_percentage', pys.abs(calculateDifferencePercentageInArea(total.area, total.areaCalculated)))

total = total.withColumn('area_differnce_percentage', pys.abs(calculateDifferencePercentageInArea(total.areaConverted, total.areaCalculated)))

In [0]:
%pyspark
total.show()

In [0]:
%pyspark
sampleMain = sqlContext.read.json("NonExploded_New_data_sample_event.json/*.json")


In [0]:
%pyspark
print sampleMain.count()
sampleMain.printSchema()

In [0]:
%pyspark
sample = sampleMain.select('sample_event.id', 'sample_event.associations.`agrian.field`', 'sample_event.sample_event_type').toDF('sample_event_id', 'field_id_temp', 'event_type')
sample = sample.groupBy('field_id_temp').agg(pys.count("event_type").alias("sample_event_count"))
sample = sample.withColumn("field_id_temp",sample.field_id_temp[0])



In [0]:
%pyspark
fieldMain = total.join(sample, sample.field_id_temp == total.id, 'left_outer').drop('field_id_temp')


In [0]:
%pyspark
fieldMain.printSchema()

In [0]:
%pyspark
grower = sqlContext.read.json("NonExploded_New_data_grower.json/*.json")


In [0]:
%pyspark
countries = sqlContext.read.json("country/*.json")
countries = countries.select('country.id','country.alpha3')


In [0]:
%pyspark
growers = grower.select('grower.id', 'grower.address.country_id').withColumnRenamed('id','grower_id_temp')
growerToCountryMap = growers.join(countries, growers.country_id==countries.id).drop('country_id', 'id').withColumnRenamed('alpha3','country')
growerToCountryMap.show()
#Free the memory!
growers = None
countries = None

In [0]:
%pyspark
fieldMain = fieldMain.withColumn("grower_id", pys.explode(fieldMain.grower_id))
fieldMain = fieldMain.withColumn("farm_id", pys.explode(fieldMain.farm_id))

In [0]:
%pyspark
fieldMain = fieldMain.join(growerToCountryMap, growerToCountryMap.grower_id_temp == fieldMain.grower_id, 'left_outer').drop('grower_id_temp')

In [0]:
%pyspark
fieldMain.printSchema()

In [0]:
%pyspark
fieldMain.show()

In [0]:
%pyspark
fieldMain2 = fieldMain.where("areaConverted != '0'").where("areaConverted != 0").where("areaConverted != 'None'")

In [0]:
%pyspark
print fieldMain.count()
print fieldMain.where("area_differnce_percentage > 20").count()

In [0]:
%pyspark
fieldMain2 = fieldMain.filter("area_differnce_percentage < 20")
fieldMain3 = fieldMain.filter("area_differnce_percentage >= 20")

temp = fieldMain2.drop('state_id')
temp2 = fieldMain3.drop('state_id')

In [0]:
%pyspark
#Filtered fields with % area error less than 20
temp.describe().show()

In [0]:
%pyspark
#Filtered fields with % arae error more than 20
temp2.describe().show()

In [0]:
%pyspark
fieldMain.printSchema()

In [0]:
%pyspark
#Get fields count on Canada
print temp.where("country = 'CAN'").count()

In [0]:
%pyspark
boundaryPresent = temp.where('isBoundaryPresent')

print boundaryPresent.select(sum('areaConverted')).show()

In [0]:
%pyspark
sampleEventPresent = temp.where("sample_event_count >= 1")
print sampleEventPresent.select(sum('areaConverted')).show()

In [0]:
%pyspark
event = sqlContext.read.json("NonExploded_New_data_event.json/*.json")


In [0]:
%pyspark
print event.count()
event = event.where(col('event.deleted_at').isNull()) 
print event.count()

In [0]:
%pyspark
event.printSchema()

In [0]:
%pyspark
eventTypes = event.select('event.json.event_type').distinct().collect()
string = ""
for i in eventTypes:
    string += str(i[0])
    string += " , "
    
print string

In [0]:
%pyspark
event2 = event.groupBy('event.associations.`agrian.field`').agg(pys.count('event.id').alias("event_count"))
event2 = event2.select( '`agrian.field`', "event_count").toDF('fieldId', 'event_count')
event2.printSchema()
dataWithEvents = temp.join(event2, temp.id == event2.fieldId[0], 'left_outer')

In [0]:
%pyspark
dataWithEvents.printSchema()

In [0]:
%pyspark
EventPresent = dataWithEvents.where("event_count >= 1")
print EventPresent.select(sum('areaConverted')).show()

In [0]:
%pyspark
dataWithEvents.describe().show()

In [0]:
%pyspark
fieldMain2.registerTempTable("fields")
sqlContext.cacheTable("fields")

In [0]:
%pyspark
dataWithEvents = dataWithEvents.where('areaConverted < 200000')

In [0]:
%pyspark
dataWithEvents.describe().show()

In [0]:
%pyspark
crop_plan = sqlContext.read.json("NonExploded_New_data_crop_plan.json/*.json")



In [0]:
%pyspark
crop_plan.printSchema()
print crop_plan.count()
crop_plan = crop_plan.where(col('crop_plan.deleted_at').isNull())
print crop_plan.count()

In [0]:
%pyspark
crop_plan = crop_plan.select('crop_plan.id', 'crop_plan.associations.`agrian.field`').toDF('crop_id', 'field_id')
crop_plan = crop_plan.groupBy('field_id').agg(pys.count('crop_id').alias("cropPlan_count"))
crop_plan.printSchema()


In [0]:
%pyspark

dataWithEventsAndCropPlan = dataWithEvents.join(crop_plan , crop_plan.field_id[0] ==  dataWithEvents.id, 'left_outer'    )

In [0]:
%pyspark
dataWithEventsAndCropPlan.describe().show()


In [0]:
%pyspark
#Getting Field id with max area
from pyspark.sql import functions as F
fields.select( F.max('field.area')).show()

data = fields.select('field.id').where("field.area >= 1.661E8 ").collect()
print data[0]


In [0]:
%pyspark
dataWithEventsAndCropPlan = dataWithEventsAndCropPlan.drop('fieldId').drop('field_id')

In [0]:
%pyspark
field_events = sqlContext.read.json("NonExploded_New_data_field_event.json")

In [0]:
%pyspark
field_eventsMin = field_events.select('field_event.id', 'field_event.field_id', 'field_event.area').toDF('id', 'fieldId', 'areaFieldEvent')
field_eventsMin.printSchema()

In [0]:
%pyspark
print field_eventsMin.select(sum('areaFieldEvent')).show()
print "Number of fields with field Events " + str(field_eventsMin.groupBy('fieldId').count().show())

In [0]:
%pyspark
print "Number of fields with field Events " + str(field_eventsMin.groupBy('fieldId').count().show())

In [0]:
%pyspark
fieldEventsWithCount = field_eventsMin.groupBy('fieldId').agg(pys.count('id').alias("fieldEvents_count"))

In [0]:
%pyspark
dataWithAllCounts = dataWithEventsAndCropPlan.join(fieldEventsWithCount, fieldEventsWithCount.fieldId == dataWithEventsAndCropPlan.id, 'left_outer').drop('fieldId')

In [0]:
%pyspark
print dataWithAllCounts.where("event_count >= 1").select(sum('areaConverted')).show()
print dataWithAllCounts.where("sample_event_count >= 1").select(sum('areaConverted')).show()
print dataWithAllCounts.where("cropPlan_count >= 1").select(sum('areaConverted')).show()
print dataWithAllCounts.where("fieldEvents_count >= 1").select(sum('areaConverted')).show()
print dataWithAllCounts.select(sum('areaConverted')).show()

In [0]:
%pyspark
dataWithAllCounts.describe().show()

In [0]:
%pyspark
dataWithEventsAndCropPlan.registerTempTable("fields")
sqlContext.cacheTable("fields")

In [0]:
%sql

select 
    case 
        when areaConverted between 0 and 1000 then '0-1000' 
        when areaConverted between 1000 and 10000 then '1000-10000'
        when areaConverted between 10000 and 20000 then '10000-20000'
        when areaConverted between 20000 and 80000 then '20000-80000'
        
        else '80000+'
    end as area_band, 
    count(*) 
from fields
group by 1 